# Run workflow

In [1]:
import os
os.chdir('..')
import numpy as np
from workflow_ephys.pipeline import lab, subject, Session, probe, ephys

Connecting root@localhost:3306


## Insert new "ClusteringParamSet" for Kilosort - in the following table:
+ ephys.ClusteringParamSet

### Define Kilosort parameters

The following parameters' values are set based on Kilosort StandardConfig file:


    https://github.com/MouseLand/Kilosort/tree/main/configFiles


In [2]:
params_ks = {
    "fs": 30000,
    "fshigh": 150,
    "minfr_goodchannels": 0.1,
    "Th": [10, 4],
    "lam": 10,
    "AUCsplit": 0.9,
    "minFR": 0.02,
    "momentum": [20, 400],
    "sigmaMask": 30,
    "ThPr": 8,
    "spkTh": -6,
    "reorder": 1,
    "nskip": 25,
    "GPU": 1,
    "Nfilt": 1024,
    "nfilt_factor": 4,
    "ntbuff": 64,
    "whiteningRange": 32,
    "nSkipCov": 25,
    "scaleproc": 200,
    "nPCs": 3,
    "useRAM": 0
}

### Insert Kilosort parameters

In [3]:
ephys.ClusteringParamSet.insert_new_params(
    'kilosort2', 0, 'Spike sorting using Kilosort2', params_ks)

## Ingestion of subjects, sessions, probes data
Extract user-specified information from `/user_data/subjects.csv` and `/user_data/sessions.csv` and insert into corresponding tables:
+ subject.Subject
+ Session
+ probe.Probe
+ ephys.ProbeInsertion

In [4]:
from workflow_ephys import ingest

In [6]:
ingest.ingest_subjects()


---- Insert 4 entry(s) into subject.Subject ----


In [7]:
ingest.ingest_sessions()

Loading Open-Ephys: reading settings...
Decoding data from  binary  format
Reading oebin file
Loading Open-Ephys: reading settings...
Decoding data from  binary  format
Reading oebin file

---- Insert 0 entry(s) into experiment.Session ----

---- Insert 0 entry(s) into experiment.Session.Directory ----

---- Insert 0 entry(s) into probe.Probe ----

---- Insert 0 entry(s) into ephys.ProbeInsertion ----

---- Successfully completed workflow_imaging/ingest.py ----


## Trigger autoprocessing of the remaining ephys pipeline

In [8]:
from workflow_ephys import populate

In [9]:
populate.populate()

EphysRecording: 0it [00:00, ?it/s]


---- Populate ephys.EphysRecording ----

---- Populate ephys.LFP ----



LFP:   0%|                                                                                       | 0/9 [00:00<?, ?it/s][Errno 2] No such file or directory: 'F:\\U24\\workflow_ephys_data\\subject1\\session1\\1\\dl56_g0_tcat.imec1.lf.bin'
[Errno 2] No such file or directory: 'F:\\U24\\workflow_ephys_data\\subject2\\session1\\SC035_010720_g0_imec0\\SC035_010720_g0_tcat.imec0.lf.bin'
[Errno 2] No such file or directory: 'F:\\U24\\workflow_ephys_data\\subject2\\session1\\SC035_010720_g0_imec2\\SC035_010720_g0_tcat.imec2.lf.bin'
[Errno 2] No such file or directory: 'F:\\U24\\workflow_ephys_data\\subject2\\session2\\SC035_011020_g0_imec0\\SC035_011020_g0_tcat.imec0.lf.bin'
[Errno 2] No such file or directory: 'F:\\U24\\workflow_ephys_data\\subject3\\session1\\SC038_111919_g0_imec0\\SC038_111919_g0_tcat.imec0.lf.bin'
[Errno 2] No such file or directory: 'F:\\U24\\workflow_ephys_data\\subject3\\session1\\SC038_111919_g0_imec1\\SC038_111919_g0_tcat.imec1.lf.bin'
[Errno 2] No such file or direct


---- Populate ephys.Clustering ----

---- Populate ephys.Waveform ----


## Insert new ClusteringTask to trigger ingestion of clustering results

Clustering is performed on the `EphysRecording` level, once clustering job has been completed, an entry in the `ClusteringTask` needs to be added to trigger the ingestion of the clustering results. Two pieces of information need to be specified:
+ the `paramset_idx` used for the clustering job
+ the output directory storing the clustering results

In [11]:
ephys.EphysRecording * ephys.EphysRecording.EphysFile

subject,session_datetime,insertion_number,file_path filepath relative to root data directory,electrode_config_hash,acq_software,sampling_rate (Hz)
subject1,2018-11-22 18:51:26,1,subject1/session1/1/dl56_g0_tcat.imec1.ap.meta,5fd1d080-c2ec-4b8f-9114-cc67642632bc,SpikeGLX,30000.0
subject2,2020-01-07 11:17:23,0,subject2/session1/SC035_010720_g0_imec0/SC035_010720_g0_tcat.imec0.ap.meta,d8fb7e4f-131c-1d67-50eb-124430ad2859,SpikeGLX,29999.5
subject2,2020-01-07 11:17:23,1,subject2/session1/SC035_010720_g0_imec1/SC035_010720_g0_tcat.imec1.ap.meta,fefd1aa0-3001-c5cc-4297-1edd60395c41,SpikeGLX,29999.4
subject2,2020-01-07 11:17:23,2,subject2/session1/SC035_010720_g0_imec2/SC035_010720_g0_tcat.imec2.ap.meta,d8fb7e4f-131c-1d67-50eb-124430ad2859,SpikeGLX,29999.7
subject2,2020-01-10 14:28:08,0,subject2/session2/SC035_011020_g0_imec0/SC035_011020_g0_tcat.imec0.ap.meta,d8fb7e4f-131c-1d67-50eb-124430ad2859,SpikeGLX,29999.5
subject2,2020-01-10 14:28:08,2,subject2/session2/SC035_011020_g0_imec2/SC035_011020_g0_tcat.imec2.ap.meta,fefd1aa0-3001-c5cc-4297-1edd60395c41,SpikeGLX,29999.4
subject3,2019-11-19 11:51:24,0,subject3/session1/SC038_111919_g0_imec0/SC038_111919_g0_tcat.imec0.ap.meta,d8fb7e4f-131c-1d67-50eb-124430ad2859,SpikeGLX,29999.4


In [15]:
import pathlib
from workflow_ephys.paths import get_ephys_root_data_dir

root_dir = pathlib.Path(get_ephys_root_data_dir())

In [20]:
for ephys_rec_key in (ephys.EphysRecording - ephys.ClusteringTask).fetch('KEY'):
    ephys_file = root_dir / (ephys.EphysRecording.EphysFile & ephys_rec_key).fetch('file_path')[0]
    recording_dir = ephys_file.parent
    kilosort_dir = next(recording_dir.rglob('spike_times.npy')).parent
    ephys.ClusteringTask.insert1({**ephys_rec_key,
                                  'paramset_idx': 0,
                                  'clustering_output_dir': kilosort_dir.as_posix()})

In [21]:
ephys.ClusteringTask()

subject,session_datetime,insertion_number,paramset_idx,clustering_output_dir clustering output directory relative to root data directory
subject1,2018-11-22 18:51:26,1,0,F:/U24/workflow_ephys_data/subject1/session1/1/imec_ww2_ks2
subject2,2020-01-07 11:17:23,0,0,F:/U24/workflow_ephys_data/subject2/session1/SC035_010720_g0_imec0/imec0_ks2
subject2,2020-01-07 11:17:23,1,0,F:/U24/workflow_ephys_data/subject2/session1/SC035_010720_g0_imec1/imec1_ks2
subject2,2020-01-07 11:17:23,2,0,F:/U24/workflow_ephys_data/subject2/session1/SC035_010720_g0_imec2/imec2_ks2
subject2,2020-01-10 14:28:08,0,0,F:/U24/workflow_ephys_data/subject2/session2/SC035_011020_g0_imec0/imec0_ks2
subject2,2020-01-10 14:28:08,2,0,F:/U24/workflow_ephys_data/subject2/session2/SC035_011020_g0_imec2/imec2_ks2
subject3,2019-11-19 11:51:24,0,0,F:/U24/workflow_ephys_data/subject3/session1/SC038_111919_g0_imec0/imec0_ks2
